In [453]:
from tools import analysistools as atools
import nanoparticle
from nanoparticle import Ligand, NanoParticle
import math
from membranesimulation import MembraneSimulation
import numpy as np
from tools import icosatiler
from tools import vectools
from tools import misctools
import k3d
import os
import random

In [408]:
mag = lambda v : np.sqrt(np.sum([i*i for i in v]))

In [457]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [410]:
def norm(v):
    m = mag(v)
    vN = [float(i)/float(m) for i in v]
    return vN

In [385]:
def crossProd(u,v):
    return ((u[1]*v[2]-u[2]*v[1]),(u[2]*v[0]-u[0]*v[2]),(u[0]*v[1]-u[1]*v[0]))

In [386]:
def makeParticleModelFromPhenome(particle):
    points = []
    plot = k3d.plot()
    i = 1
    maxEps = 0
    for l in particle.ligands:
        if l.eps>maxEps:
            maxEps = l.eps
    for l in particle.ligands:
        if l.eps > 0.0:
            lx = l.rad*np.sin(l.polAng)*np.cos(l.aziAng)
            ly = l.rad*np.sin(l.polAng)*np.sin(l.aziAng)
            lz = l.rad*np.cos(l.polAng)
            lc = '0x' + ('#%02x%02x%02x' % (0, int((l.eps/maxEps)*255.0), 0))[1:]
            
            plot += k3d.points([lx,ly,lz],point_size=1,color=int(lc, 16))
            #plot += k3d.points([lx,ly,lz],point_size=1,color=int(0x00ffff*(l.eps/maxEps)))
            plot += k3d.text(str(i),[lx,ly,lz],color=0xffffff)
            i+=1
    plot += k3d.points([0,0,0],point_size=7,color=0xff0000)
    return plot

In [387]:
def buildCoveredParticle(eps,rad):
    particle = NanoParticle()
    lPos = icosatiler.cover72SpherePolar(rad)
    for v in lPos:
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [388]:
def spaceIsOccupied(particle,v):
    for l in particle.ligands:
        d = atools.greatArcDist((l.polAng,l.aziAng),(v[1],v[2]),particle.sig)
        if abs(d) < l.size:
            return True
    return False

In [475]:
def getClosestIcos(particle,v):
    lPos = icosatiler.cover72SpherePolar(particle.sig)
    closest = v
    minD = 1e8
    for l in lPos:
        v1 = icosatiler.sphPol2Crt(l) 
        v2 = icosatiler.sphPol2Crt(v)
        d = np.sum([j*j for j in np.subtract(v1,v2)])
        if abs(d) < minD:
            minD = d
            closest = l
    return closest

In [390]:
# lPos = icosatiler.cover72SpherePolar(4)
# dists = []
# minD = 1e8
# for v in lPos:
#     for l in lPos:
#         if l != v:
#             v1 = icosatiler.sphPol2Crt(l) 
#             v2 = icosatiler.sphPol2Crt(v)
#             d = np.sum([j*j for j in np.subtract(v1,v2)])
#             if abs(d) < minD:
#                 minD = d
#     dists.append(np.sqrt(minD))
# print np.mean(dists)

In [391]:
def addLigand(particle,eps,v,lock=False):
    if lock:
        v = getClosestIcos(particle,v)
    if not spaceIsOccupied(particle,v):
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [392]:
def addBelt(particle,eps,rad,n,phi,lock=False):
    step = np.pi*2/float(n)
    for i in range(n):
        v = (rad,step*i,phi)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [393]:
def getArcAngle(dist,rad):
    return dist/rad

In [426]:
def addRing(particle,eps,rad,pos,n,spacing,lock=False):
    step = np.pi*2/float(n)
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    for i in range(n):
        tD = getArcAngle(spacing,rad)
        tP = step*i
        v = (rad,tD,tP)
        vC = icosatiler.sphPol2Crt(v)
        v1 = icosatiler.sphPol2Crt((rad,0,0))
        v2 = icosatiler.sphPol2Crt((rad,pos[0],pos[1]))
        v3 = crossProd(v1,v2)
        if np.sum([j*j for j in v3])== 0.0:
            #it's parallel!
            particle = addLigand(particle,eps,v,lock)
            continue
        dp = np.dot(v1,v2)
        #the magic sauce :)
        ang = np.arctan2(mag(np.cross(v1,v2)), np.dot(v1,v2))
        rM = vectools.buildERMatrix(v3, ang)
        vC = np.dot(rM,vC)
        v = icosatiler.crt2SphPol(vC)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [492]:
def addPatch(particle,eps,rad,pos,n,spacing,lock=False):
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    particle = addLigand(particle,eps,(rad,pos[0],pos[1]),lock)
    particle = addRing(particle,eps,rad,pos,n,spacing,lock)
    return particle

In [ ]:
def addMesh(particle,eps,rad,spacing,lock=False):
    

In [497]:
#makeParticleModelFromPhenome(buildCoveredParticle(6,4))

In [507]:
p = NanoParticle()
m = 3
step = np.pi*2/float(m)
for i in range(m):
    p = addPatch(p,6,4,(i*step,np.pi/2.0),6,1.6,lock=True)
p = addPatch(p,6,4,(np.pi/2.0,0),6,1.6,lock=True)
p = addPatch(p,6,4,(3*np.pi/2.0,0),6,1.6,lock=True)
p = addBelt(p,6,4,29,0,lock=True)

In [508]:
for l in p.ligands:
    if getClosestIcos(p,(l.rad,l.polAng,l.aziAng)) != (l.rad,l.polAng,l.aziAng):
        print (l.rad,l.polAng,l.aziAng), getClosestIcos(p,(l.rad,l.polAng,l.aziAng))

In [509]:
makeParticleModelFromPhenome(p)

Plot(antialias=True, background_color=16777215, grid=[-1, -1, -1, 1, 1, 1], height=512, object_ids=[120991866256, 120992189328, 120992189520, 120991863248, 120991865680, 120991865808, 120992190096, 120987416720, 120992188944, 120991866192, 120992395856, 120992189392, 120992395984, 120991864080, 120992397072, 120992395920, 120992397136, 120992189712, 120992396624, 120992396304, 120992396496, 120992395536, 120992396560, 120992397008, 120992397200, 120992398608, 120992396816, 120992397520, 120992397968, 120992397712, 120992397776, 120992397584, 120992396688, 120992397328, 120992396368, 120992555472, 120992555216, 120992555856, 120992556176, 120992555728, 120992398416, 120992399120, 120992556816, 120992395664, 120992557264, 120992557776, 120992398736, 120992556496, 120992558224, 120992557712, 120992555920, 120992557200, 120992556944, 120992555152, 120992558352, 120992557328, 120992558160, 120992558992, 120992699024, 120992557136, 120992558032, 120992556688, 120992699344, 120992699152, 120992559056, 120992699472, 120992558416, 120992698640, 120992698960, 120992699920, 120992700816, 120992700624, 120992700944, 120992699216, 120992700368, 120992701264, 120992700688, 120992701904, 120992701456, 120992700752, 120992699856, 120992701072, 120992701776, 120992702224, 120992188688])

In [510]:
RUNTIME = 25000
TIMESTEP = 0.01
OUTDIR = "/Users/joelforster/Projects/optidb/models/out/"
RUNDIR = "/Users/joelforster/Projects/optidb/models/run/"
TEMPLATEDIR = "/Users/joelforster/Projects/optihedron/mem/template"
TEMPLATEDATAPATH = os.path.join(TEMPLATEDIR,'data.template')
TEMPLATEINPUTPATH = os.path.join(TEMPLATEDIR,'in.template')

ensure_dir(OUTDIR)
ensure_dir(RUNDIR)

sim = MembraneSimulation(
        'model_'+misctools.randomStr(10),
        p,
        RUNTIME,
        TIMESTEP,        
        OUTDIR,
        RUNDIR,
        TEMPLATEDATAPATH,
        TEMPLATEINPUTPATH,
        rAxis=vectools.randomUnitVector(),
        rAmount=random.uniform(0.3141,3.141)        
        )
sim.saveFiles()

saved files: /Users/joelforster/Projects/optidb/models/run/model_WHPC5DADA4_data.data, /Users/joelforster/Projects/optidb/models/run/model_WHPC5DADA4_script.in
